# Data Analyst Certificate - Sales Strategies Analysis

**New Product Sales Methods**

Six weeks ago we launched a new line of office stationery. Despite the world becoming increasingly digital, there is still demand for notebooks, pens and sticky notes.

Our focus has been on selling products to enable our customers to be more creative, focused on tools for brainstorming. We have tested three different sales strategies for this, targeted
email and phone calls, as well as combining the two.

- **Email**: Customers in this group received an email when the product line was launched, and a further email three weeks later. This required very little work for the team.
- **Call**: Customers in this group were called by a member of the sales team. On average members of the team were on the phone for around thirty minutes per customer.
- **Email and call**: Customers in this group were first sent the product information email, then called a week later by the sales team to talk about their needs and how this new product
may support their work. The email required little work from the team, the call was around ten minutes per customer.

The sales rep has pulled some data from their sales tracking system for us. They haven’t included numbers for how much time was spent on each customer, but there may be some other useful customer information in here.

|Column Name | Details|
|------------|--------|
|`week`             |Week sale was made, counted as weeks since product launch|
|`sales_method`     |Character, which of the three sales methods were used for that customer|
|`customer_id`      |Character, unique identifier for the customer|
|`nb_sold`          |Numeric, number of new products sold|
|`revenue`          |Numeric, revenue from the sales, rounded to 2 decimal places.|
|`years_as_customer`|Numeric, number of years customer has been buying from us (company founded in 1984)|
|`nb_site_visits`   |Numeric, number of times the customer has visited our website in the last 6 months|
|`state`            |Character, location of the customer i.e. where orders are shipped|

# Data Validation

Before cleaning and validation, the dataset contains 15000 rows and 8 columns. After initial exploration, three main issues were found:
- There are typos in the `sales_method` column.
- There are forbidden values in the `years_as_customer` column.
- There are missing values in the `revenue` column.

## Cleaning `sales_method` column

The unique values found in the original dataset were *Email, Email + Call, Call, em + call, email*. I replaced entries containing *em + call* and *email* with *Email + Call* and *Email*, respectively.

By the end of the transformations, the `sales_method` column contains only the categories: **Email, Email + Call** and **Call**.

## Addressing forbidden values in `years_as_customer` column

The company was founded in 1984, so **there cannot be users with more than 41 years as customers**. I found 2 rows that did not match this criterion, so I decided to remove them.

## Addressing missing values in `revenue` column

The `revenue` column contains 1074 null values, which represent about 7% of the total number of records. That is above the 3% usual threshold for missing data.

To address this issue, I have explored the revenue distribution per sales method and applied different imputation approaches for each category.

### Addressing missing values: *Call* method

For the *Call* method, missing values represent slightly more than 3%. It presents a right-skewed distribution; therefore, **I decided to replace missing values with the median value** of the revenue within the *Call* method group.

### Imputation by sub-groups: *Email* method

For the *Email* method, missing values represent above 7% of data. Some imputation is needed.

The revenue distribution for the *Email* method shows a right-skewed multimodal distribution, displaying two peaks at 85 and 105. **Applying mean or mode imputation would not be appropriate in this case.**

![email_distr_1](email_distr_1.png)


After further exploration, I observed a relationship between `week` and `revenue` that could serve as a good criterion for a sub-group imputation: **for revenues above 100, the average weeks since launch is 3.84, while for revenues below 100, it is 1.63 weeks.** 

Therefore, I decided to create two different groups within customers of the *Email* method, splitting by number of weeks since launch. The following graph shows the two different distributions obtained.

![email_distr_2](email_distr_2.png)

Missing values in each group were replaced by their corresponding median value. 

### Imputation by sub-groups: *Email + Call* method

The *Email + Call* case is similar to the *Email* one. Missing values represent more than 13% of data, and revenue presents a multimodal distribution.

![email_call_distr](email_call_distr.png)


I split data into 4 subgroups by their revenue values, each of them being normally distributed. These groups correlate 
with the `nb_sold` column, as displayed in the following graph.

![email_call_2](email_call_2.png)

Therefore, I have gathered the *Email + Call* entries into 4 groups by their `nb_sold` values and applied mean imputation to missing values in each group.

## Definitive dataset for analysis

**After data validation, the dataset contains 14998 rows and 8 columns**:

- `week`: 6 unique numeric values, from 1 to 6, as informed in the description. No cleaning needed. No missing values.
- `sales_method`: 3 categories as informed in the description. Cleaning performed as described before. No missing values.
- `customer_id`: 14998 unique categories, as informed in the description. No cleaning needed. No missing values.
- `nb_sold`: numeric values from 7 to 16, as informed in the description. No cleaning needed. No missing values.
- `revenue`: numeric values from 32.54 to 238.32, rounded to 2 decimal places, as informed in the description. Missing values imputed as described before.
- `years_as_customer`: numeric values from 0 to 41, as informed in the description. Cleaning performed as described before. No missing values.
- `nb_site_visits`: numeric values from 12 to 41, as informed in the description. No cleaning needed. No missing values.
- `state`: 50 unique categories, as informed in the description. No cleaning needed. No missing values.

# Exploratory Analysis

## How many customers were there for each approach?

From the dataset tracking sales from the last 6 weeks, I found that **there were 7465 customers for the *Email* approach (50% of total), 4961 for the *Call* approach (33% of total) and 2572 for the *Email and Call* approach (17% of total)**.

![number_customers](number_customers.png)

The number of customers for the *Email* approach is almost equal to the sum of the other two methods. It is worth noting that the second most time-demanding method, which is *Email and Call*, presents almost half the sales of the *Call* method, the most demanding for the sales team. 

Since our dataset only tracks customers who made a purchase and not the total number of addressed customers, we cannot draw any further conclusions regarding the effectiveness of each approach. Given that the *Email* method is the least demanding for the sales team, it is very likely that significantly more customers were addressed through this approach than through the others during the same 6 weeks. 

Moreover, after exploring the total revenue per sales method, it is by the *Email* approach that the company achieved the highest total revenue. However, the *Email and Call* total revenue is about 65% of the *Email*'s, despite having only 35% of its customers.

![total_revenue](total_revenue.png)

## What does the spread of the revenue look like overall? And for each method?

Overall revenue shows a multimodal distribution, with the majority of revenues lying below 150, but with 3 distinct peaks occurring around 50, 90 and 180. It presents a significant spread, **ranging from 32.54 to 238.32**.

![rev_distr](rev_distr.png)

This multimodal behaviour coresponds to distinct sales methods, so extreme values above 150 cannot be classified as outliers.

When comparing the spread of the revenue for each method, the IQR is considered the most significant measure. **The *Email + Call* method performed the best, achieving revenues from 155 to 190; the *Email* method ranked second, with revenues ranging from 85 to 105, while the *Call* method performed the worst, with revenues between 40 and 55**.

![rev_per_method](rev_per_method.png)

## Difference in revenue over time for each method

The dataset tracks sales after the launch of a new line of office stationery. The column `week` records the number of weeks since launch, ranging from 1 to 6. I will use this variable to **assess the evolution of average revenue over time**.

![rev_over_time](rev_over_time.png)

Some conclusions:
- All three methods display a growing average revenue over time.
- **The *Call* method performed the worst**, while also being the most time-demanding for the sales team.
- The difference between *Email* and *Call* average revenues remained almost constant: both have the same growth rate in average revenue.
- **The *Email + Call* method performed the best by far**, achieving the highest average revenue every week and showing the highest growth rate over time.

Another interesting metric is the time evolution of the total revenue obtained with each method. 

![total_rev_evol](total_rev_evol.png)

Recall that the *Email* customers contributed the largest amount of revenue, but most of it was achieved in the first week: **total revenue for the *Email* method decreases rapidly over time**, although its average revenue per customer is slightly increasing. 

On the other hand, **the total revenue from *Email + Call* increases over time and has the highest revenue per customer every week. Moreover, its average revenue per customer is rapidly increasing throughout the weeks**.

## Customers' Purchasing History and Number of Products Sold for each method

In order to better characterize customers in each group, it is worth exploring the distribution of `years_as_customer` for each method.

![years_distr](years_distr.png)

Although `years_as_customer` ranges from 0 to 41 years, **most sales are concentrated among users with less than 10 years as customers**. This distribution is the same for every method.

Regarding the number of products sold with each method, the results are similar but **the *Email + Call* approach performed the best, ranging from 10 to 13 with a median of 12 products sold to each customer**. Products sold within the *Call* method ranged from 8 to 11, and similar results were obtained for the *Email* approach, with the number of products sold ranging between 9 and 11. 

![products_per_met](products_per_met.png)

# Business Metric

Given that the sales team aims to sell their products effectively, **I would recommend using the total revenue** as our main metric to monitor. 

Based on the latest data, and after applying the three different methods for approaching customers for 6 weeks, the total revenue was $1,434,725.98.

# Summary and Recommendations

There are some conclusions to draw from the sales data analysis:
- **The *Call* method**, which requires significant effort from the team, **performed the worst in key metrics**, such as average revenue per customer and overall revenue.
- **The *Email* method**, the easiest for the sales team, **worked well in volume, attracting the highest number of customers and displaying the highest overall revenue** after the first 6 weeks of selling the products. 
- Nevertheless, the *Email* method loses scalability over time and does not achieve the highest average revenue per customer.
- **The *Email + Call* method is attractive to fewer customers, but generates the highest average revenue**: customers who made a purchase after being approached by the *Email + Call* method tend to spend the most and to buy more items on average.
- Moreover, the *Email + Call* method shows the highest growth in revenue (overall and in average revenue per customer) over time.

For these reasons, I recommend following the next steps:

✅ **Discontinue approaches using the *Call* method**. 

✅ **Continue using the *Email* method** since it allows for broader reach and requires little effort from the team.

✅ **Continue using the *Email + Call* method**, since it attracts high-value customers and scales well over time.

✅ **Focus on customers with less than 10 years** of buying from the company.

To improve future analyses, I recommend:
- Tracking the total number of customers contacted by the sales team to assess how effective each method was in actually producing a sale.
- Recording what products were sold to each customer to identify customer preferences.
- Logging how much time the team spent on each customer to evaluate efficiency.